# Superstore Sales: Clustering Process

## Imports

In [2]:
import warnings
warnings.filterwarnings('ignore')

from helper_funcs import *
from sqlalchemy import create_engine, types
from sqlalchemy.types import *

import numpy as np
import scipy.stats as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans

In [3]:
# SQLAlchemy Engine
engine = create_engine(generate_url())

## General EDA

In [4]:
df = pd.read_sql_table(table_name='features', con=engine.connect())
df.head(5)

ValueError: Table features not found

In [ ]:
df.shape

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
sns.countplot(data=df, x='Region', hue='Segment', ax=ax, palette=sns.color_palette('hls', 3))
ax.set_ylabel('Number of Orders')
ax.set_title('Total Orders by Region and Segment')
fig.tight_layout()
plt.show()

**REMARK**: Similar to our Copier inquery from out Categorical Analysis, West- and East-coast consumers lead total orders (volume). Also of note, Corporate orders outpace Home Office volume in all four regions.

In [ ]:
# Selecting the 10 States with the most orders, aggregate Profit, and sort.
T10 = df['State'].value_counts()[:10].index.tolist()
top_state_df = df[df['State'].isin(T10)][['State', 'Profit']].groupby(by='State', as_index=False).sum().sort_values(by='Profit', ascending=False)

In [ ]:
# Plot top_state_df
fig, ax = plt.subplots(figsize=(8, 6))
sns.barplot(data=top_state_df, x='Profit', y='State', orient='h', ax=ax)
ax.set_title('Top-Performing US States')
fig.tight_layout()
plt.show()

**REMARK**: Unsurprisingly, California and New York compete for most Profitable (which here relates to most orders. Washington makes up roughly half of either, but Texas doesn't even make the Top 5.

## KMeans Clustering

SKLearn's OneHotEncoder may work too, but for our purposes, we're gonna do this quick and dirty.

In [ ]:
# Detect columns as either numeric or categorical
num_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_cols = df.select_dtypes(include='object').columns.tolist()

print('Categorical Features:', cat_cols)
#cat_cols.tolist()

In [ ]:
# Encode categorical data with dummy variables, normalize numerical features, then concatenate back together.
norm_df = pd.concat([df[num_cols].apply(st.zscore), pd.get_dummies(df[cat_cols])], axis=1)
norm_df.info()

In [ ]:
# Correct a TypeError
norm_df.columns = norm_df.columns.astype(str)

In [ ]:
inertial_vals = []

K_vals  = range(1, 10)
for k in K_vals:
    kmeans = KMeans(k)
    kmeans.fit(norm_df)
    inertial_vals.append(kmeans.inertia_)

plt.plot(K_vals, inertial_vals, marker='o')
plt.title('Elbow Method for Optimal k')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Sum of Squared Distances (Inertia)')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=5, random_state=101).fit(norm_df)
df['Cluster'] = kmeans.labels_

In [ ]:
df['Cluster'].value_counts()

In [ ]:
# Replace features to include 'Cluster'
df.to_sql(name='features', con=engine, if_exists='append', index=False, chunksize=1000, method='multi')